In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np

# Camera calibration

We'll begin camera calibration with loading given images. 

In [ ]:
import glob
def load_images(load_path):
    image_paths = glob.glob(load_path)
    images = [plt.imread(path) for path in image_paths]
    return images
        
def display_grid(images, n_col, title, cmap=None):
    plt.close('all')
    fig, ax_arr = plt.subplots(len(images)//n_col, n_col,figsize=(15,7))
    fig.tight_layout()
    fig.suptitle(title, fontsize=30)
    for i, image in enumerate(images):
        ax = ax_arr[i // n_col, i % n_col]
        ax.imshow(image,cmap=cmap)
#         ax.set_xticks([]), ax.set_yticks([])
    plt.show()


calibration_images = load_images("camera_cal/*.jpg")
display_grid(calibration_images, 5, "Images for camera calibration")

As shown above there are 20 images. Using them we could do camera calibration.



In [ ]:
def find_corners(image, pattern_size):
    """
    Look for the corners on chessboard image given pattern size
    
    Args:
        image: cv2.image
        pattern_size: tuple
    
    Returns:
        List of corners coordinates if they are found
    """
    
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    retval, corners = cv2.findChessboardCorners(gray, pattern_size)
    return corners if retval else []

import functools

def find_calibration_parameters(images, pattern_size):
    return_corners = list(map(lambda x: find_corners(x, pattern_size), images))
    corners = list(filter(lambda x: len(x) != 0, return_corners))

    objp_size = np.prod(pattern_size), 3
    objp = np.zeros(objp_size, np.float32)
    objp[:,:2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1,2)
    objpoints = [objp] * len(corners)
    
    image_size = images[-1].shape[1::-1]
    ret, mtx, dist, *rest = cv2.calibrateCamera(objpoints, corners, image_size, None, None)
    
    return ret, mtx, dist, return_corners

def undistort_images(images, mtx, dist):
    return [cv2.undistort(image, mtx, dist) for image in images]

def transform_perspective(image, corners, pattern_size = (2,2), offset = 0):
    if len(corners) == 0:
        img = image.copy()
        cv2.putText(img, "Failed", (10, 100), cv2.FONT_ITALIC, 4.0, (0, 0, 255), 3)
        return img
    nx, ny = pattern_size
    src = np.float32([corners[0], corners[nx-1], corners[-1], corners[-nx]])
    image_size = image.shape[1::-1]
    (w, h), d = image_size, offset
    dst = np.float32([[d, d], [w - d, d], [w-d, h-d], [d, h - d]])
    M = cv2.getPerspectiveTransform(src, dst)
    return cv2.warpPerspective(image, M, image_size)

ps = (9, 6)
ret, mtx, dist, corners = find_calibration_parameters(calibration_images, pattern_size=ps)

if ret:
    undistorted_images = undistort_images(calibration_images, mtx, dist)
#     display_grid(undistorted_images, 5, "Undistorted images")

    transformed_images = [transform_perspective(image, corners[i], pattern_size=ps, offset=100) \
                          for i, image in enumerate(undistorted_images)]
    display_grid(transformed_images, 5, "Transformed images")


It's interesting to note that some images aren't helpful because the `findChessboardCorners` method cannot detect corners on them.

# Pipeline (for `test_images` folder)

In [ ]:
test_images = load_images("test_images/*.jpg")
display_grid(test_images, 4, "Test images")

In [ ]:
def undistort_and_transform_perspective(images):
    undistorted_images = undistort_images(images, mtx, dist)
    h, w, *_ = images[-1].shape
    corners = [(0.35*w, 0.7*h), (0.65*w, 0.7*h), (0, 0.9*h), (w, 0.9*h)]
    transformed_images = [transform_perspective(image, corners) \
                          for i, image in enumerate(undistorted_images)]
    display_grid(transformed_images, 4, "Transformed images")
    
undistort_and_transform_perspective(test_images)

In [ ]:
def take_sobel_op(gray, sobel_kernel = 3):
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    return sobelx, sobely

def abs_sobel_thresh(sobelx, sobely, orient='x', thresh = (0, 255)):
    
    abs_sobel = np.absolute(sobelx) if orient == 'x' else np.absolute(sobely)
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1

    return binary_output

def mag_thresh(sobelx, sobely, thresh=(0, 255)):
    abs_sobel_xy = np.sqrt(sobelx ** 2 + sobely ** 2)
    abs_sobel_xy = (abs_sobel_xy * 255 / np.max(abs_sobel_xy)).astype(np.uint8)

    binary_output = np.zeros_like(abs_sobel_xy)
    binary_output[(abs_sobel_xy >= thresh[0]) & (abs_sobel_xy <= thresh[1])] = 1

    return binary_output


def dir_threshold(sobelx, sobely, thresh=(0, np.pi/2)):
    abs_sobelx, abs_sobely = np.absolute(sobelx), np.absolute(sobely)

    a = np.arctan2(abs_sobely, abs_sobelx)
    
    binary_output = np.zeros_like(a)
    
    binary_output[(a >=thresh[0]) & (a <= thresh[1])] = 1
    return binary_output

def image_processing_pipeline(gray):
    sobelx, sobely = take_sobel_op(gray)
    r = [
        abs_sobel_thresh(orient='x', sobelx=sobelx, sobely=sobely, thresh=(35, 150)),
        abs_sobel_thresh(orient='y', sobelx=sobelx, sobely=sobely, thresh=(10, 150)),
        mag_thresh(sobelx=sobelx, sobely=sobely, thresh=(50, 200)),
        dir_threshold(sobelx=sobelx, sobely=sobely, thresh=(0.7, 1.1))
    ]
    combined = np.zeros_like(sobelx)
    combined[((r[0] == 1) & (r[1] == 1)) | ((r[2] == 1) & (r[3] == 1))] = 1
    
    return combined

def _experiment(gray):
    sobelx, sobely = take_sobel_op(gray)
    result = []
    for i in range(0, 8*10, 10):
        result.append(abs_sobel_thresh(orient='y', sobelx=sobelx, sobely=sobely, thresh=(60,60+i)))
    print(len(result))
    return result
        
#gray_test_images = [cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in test_images]
converted = [image_processing_pipeline(gray) for gray in gray_test_images]
# converted = _experiment(gray_test_images[0])
display_grid(converted, n_col=4, title="Thresholded images", cmap='gray')
# gradient_threshold_images = [gradient_threshold(img, (20, 100)) for img in gray_test_images]
# display_grid(gradient_threshold_images)